In [3]:
import torch
import os

In [4]:
# Set TORCH_HOME to your desired location
os.environ['TORCH_HOME'] = r'/mnt/d/23b6034/FYP/models'

# Now load the model
model2 = torch.hub.load('facebookresearch/detr:main', 'detr_resnet50', pretrained=True)


Using cache found in /mnt/d/23b6034/FYP/models/hub/facebookresearch_detr_main
/mnt/d/23b6034/FYP/vengeance/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/d/23b6034/FYP/vengeance/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Save individual parts of the DETR model to separate files

# Helper to create simple filenames
model_dir = os.path.join(os.environ.get('TORCH_HOME', '.'), 'detr_parts')
os.makedirs(model_dir, exist_ok=True)

paths = {
    'backbone': os.path.join(model_dir, 'detr_backbone.pth'),
    'transformer_encoder': os.path.join(model_dir, 'detr_transformer_encoder.pth'),
    'transformer_decoder': os.path.join(model_dir, 'detr_transformer_decoder.pth'),
    'head': os.path.join(model_dir, 'detr_head.pth'),
}

# Inspecting model structure (adapt these attribute names if your DETR variant differs)
# For the facebookresearch DETR, typical attributes are: backbone, transformer, and class/box heads under model or a separate DETR wrapper.
print('Model attrs:', [a for a in dir(model2) if not a.startswith('_')])

# Extract state_dicts for each part (best-effort; adjust keys according to your model)
full_sd = model2.state_dict()

backbone_sd = {k.replace('backbone.', ''): v for k, v in full_sd.items() if k.startswith('backbone.')}
transformer_encoder_sd = {k.replace('transformer.encoder.', ''): v for k, v in full_sd.items() if k.startswith('transformer.encoder.')}
transformer_decoder_sd = {k.replace('transformer.decoder.', ''): v for k, v in full_sd.items() if k.startswith('transformer.decoder.')}
# Heads often live under 'class_embed' / 'bbox_embed' or 'heads' depending on implementation
head_keys = [k for k in full_sd.keys() if ('class_embed' in k) or ('bbox_embed' in k) or k.startswith('bbox_head') or 'head' in k]
head_sd = {k: full_sd[k] for k in head_keys}

# Save parts
torch.save(backbone_sd, paths['backbone'])
torch.save(transformer_encoder_sd, paths['transformer_encoder'])
torch.save(transformer_decoder_sd, paths['transformer_decoder'])
torch.save(head_sd, paths['head'])

print('Saved parts to', model_dir)


Model attrs: ['T_destination', 'add_module', 'apply', 'aux_loss', 'backbone', 'bbox_embed', 'bfloat16', 'buffers', 'call_super_init', 'children', 'class_embed', 'compile', 'cpu', 'cuda', 'double', 'dump_patches', 'eval', 'extra_repr', 'float', 'forward', 'get_buffer', 'get_extra_state', 'get_parameter', 'get_submodule', 'half', 'input_proj', 'ipu', 'load_state_dict', 'modules', 'mtia', 'named_buffers', 'named_children', 'named_modules', 'named_parameters', 'num_queries', 'parameters', 'query_embed', 'register_backward_hook', 'register_buffer', 'register_forward_hook', 'register_forward_pre_hook', 'register_full_backward_hook', 'register_full_backward_pre_hook', 'register_load_state_dict_post_hook', 'register_load_state_dict_pre_hook', 'register_module', 'register_parameter', 'register_state_dict_post_hook', 'register_state_dict_pre_hook', 'requires_grad_', 'set_extra_state', 'set_submodule', 'share_memory', 'state_dict', 'to', 'to_empty', 'train', 'training', 'transformer', 'type', 'xp

Using cache found in /mnt/d/23b6034/FYP/models/hub/facebookresearch_detr_main
/mnt/d/23b6034/FYP/vengeance/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/d/23b6034/FYP/vengeance/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded parts into new_model (strict=False)


In [10]:
# Load parts into a fresh DETR model (separate load cell)
# Build model from local python files rather than torch.hub
import importlib
import importlib.util
import torch
import sys
from pathlib import Path
import argparse

# Try to import a builder from common local module locations.
workspace_root = Path('.')  # adjust if your project root is elsewhere
sys.path.insert(0, str(workspace_root))

candidates = [
    ('detr', 'build_detr'),
    ('detr', 'build'),
    ('models.detr', 'build_detr'),
    ('models.detr', 'build'),
    ('models', 'build_detr'),
    ('models', 'build'),
    ('Danial', 'build_detr'),
    ('Danial', 'build'),
    ('', 'build_detr'),
    ('', 'build'),
]

builder = None
builder_name = None
for mod_name, fn_name in candidates:
    try:
        if mod_name:
            mod = importlib.import_module(mod_name)
        else:
            # attempt top-level
            mod = importlib.import_module(fn_name)
        if hasattr(mod, fn_name):
            builder = getattr(mod, fn_name)
            builder_name = f"{mod_name}.{fn_name}" if mod_name else fn_name
            break
    except Exception:
        continue

if builder is None:
    # Try to locate common filenames in the workspace and import by path
    found = False
    for p in workspace_root.rglob('build*.py'):
        spec = importlib.util.spec_from_file_location('local_build', str(p))
        mod = importlib.util.module_from_spec(spec)
        try:
            spec.loader.exec_module(mod)
            if hasattr(mod, 'build') or hasattr(mod, 'build_detr'):
                builder = getattr(mod, 'build', getattr(mod, 'build_detr'))
                builder_name = str(p)
                found = True
                break
        except Exception:
            continue
    if not found:
        raise ImportError('Could not find a local DETR builder. Please provide a module with a `build` or `build_detr` function in the workspace.')

print('Using builder:', builder_name)

# The selected builder expects an `args` object with many attributes (see DETR's build signature).
# We'll construct a minimal argparse.Namespace with sensible defaults. If `model2` exists (from earlier run), try to infer some values.
# Typical DETR defaults (matching facebookresearch/detr):
# - backbone: 'resnet50'
# - device: 'cpu' or 'cuda'
# - dataset_file: 'coco' (affects num_classes)
# - hidden_dim: 256, nheads: 8, dim_feedforward: 2048, enc_layers: 6, dec_layers: 6
# - num_queries: 100
# - other training hyperparams used by build: lr_backbone, masks, dilation, aux_loss, etc.

# Attempt to infer defaults from an existing model2 when available
hidden_dim = 256
nheads = 8
enc_layers = 6
dec_layers = 6
num_queries = 100
backbone_name = 'resnet50'

if 'model2' in globals():
    try:
        if hasattr(model2, 'transformer') and hasattr(model2.transformer, 'd_model'):
            hidden_dim = int(getattr(model2.transformer, 'd_model'))
        if hasattr(model2.transformer, 'nhead'):
            nheads = int(getattr(model2.transformer, 'nhead'))
        # encoder/decoder layers counts are available on transformer implementation used here
        if hasattr(model2.transformer, 'encoder') and hasattr(model2.transformer.encoder, 'num_layers'):
            enc_layers = int(getattr(model2.transformer.encoder, 'num_layers'))
        if hasattr(model2.transformer, 'decoder') and hasattr(model2.transformer.decoder, 'num_layers'):
            dec_layers = int(getattr(model2.transformer.decoder, 'num_layers'))
        if hasattr(model2, 'num_queries'):
            num_queries = int(getattr(model2, 'num_queries'))
        # backbone detection: fallback to resnet50
        try:
            backbone_name = getattr(model2.backbone.body, 'orig_name', backbone_name)
        except Exception:
            pass
    except Exception:
        pass

# Build a namespace with the attributes the builder expects
args = argparse.Namespace(
    dataset_file='coco',
    device='cpu',
    backbone=backbone_name,
    lr_backbone=0,
    masks=False,
    dilation=False,
    hidden_dim=hidden_dim,
    dropout=0.1,
    nheads=nheads,
    dim_feedforward=2048,
    enc_layers=enc_layers,
    dec_layers=dec_layers,
    pre_norm=False,
    num_queries=num_queries,
    aux_loss=False,
    frozen_weights=None,
    set_cost_class=1,
    set_cost_bbox=5,
    set_cost_giou=2,
    bbox_loss_coef=5,
    giou_loss_coef=2,
    mask_loss_coef=1,
    dice_loss_coef=1,
    eos_coef=0.1,
)

# Build model using the args namespace
try:
    new_model = builder(args)
except TypeError:
    # Some builders return (model, ...) -> handle both cases
    out = builder(args)
    if isinstance(out, tuple):
        new_model = out[0]
    else:
        new_model = out

# If the builder returned (model, criterion, postprocessors) like in DETR, handle that
if isinstance(new_model, tuple):
    new_model = new_model[0]

# Merge saved parts into new_model
new_sd = new_model.state_dict()

# Load backbone
loaded_backbone = torch.load(paths['backbone'], map_location='cpu')
for k, v in loaded_backbone.items():
    new_sd['backbone.' + k] = v

# Load transformer encoder
loaded_te = torch.load(paths['transformer_encoder'], map_location='cpu')
for k, v in loaded_te.items():
    new_sd['transformer.encoder.' + k] = v

# Load transformer decoder
loaded_td = torch.load(paths['transformer_decoder'], map_location='cpu')
for k, v in loaded_td.items():
    new_sd['transformer.decoder.' + k] = v

# Load head (merge keys as-is where appropriate)
loaded_head = torch.load(paths['head'], map_location='cpu')
for k, v in loaded_head.items():
    # try exact key, otherwise attempt common prefixes
    if k in new_sd:
        new_sd[k] = v
    elif 'class_embed.' + k in new_sd:
        new_sd['class_embed.' + k] = v
    else:
        # best-effort: try to find a matching suffix
        matches = [nk for nk in new_sd.keys() if nk.endswith(k)]
        if matches:
            new_sd[matches[0]] = v

# Finally load into new_model with strict=False so missing/unexpected keys are allowed
new_model.load_state_dict(new_sd, strict=False)
print('Loaded parts into new_model (strict=False)')


Using builder: models.detr.build


AttributeError: 'Namespace' object has no attribute 'position_embedding'